In [1]:
import pandas as pd

train_data = pd.read_csv(r'E:\Develop\python\NLP\howls_nlp\Chatbot\자료\results\Q&A.csv', encoding = 'cp949')
print(train_data.keys())
lemma = list(train_data["LEMMA"][3023:])
ans = list(train_data['ANSWER'][3023:])
ans = [str(i)[2:-2] for i in ans]
data_pair = [[lemma[i], ans[i]] for i in range(len(lemma))]
print(data_pair[1][1])

Index(['QUESTION', 'ANSWER', 'TAGSET', 'LEMMA'], dtype='object')
미네르바는 교양필수과목이기 때문에 타교양으로 재수강 연결을 할 수 없습니다 (동일과목으로만 재수강 가능)


In [2]:
import random
import torch
import torch.nn as nn
from torch import optim

In [3]:
torch.manual_seed(0)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [4]:
SOS_token = 0
EOS_token = 1

In [5]:
class Vocab:
  def __init__(self):
    self.vocab2index = {"<SOS>": SOS_token, "<EOS>": EOS_token}
    self.index2vocab = {SOS_token: "<SOS>", EOS_token: "<EOS>"}
    self.vocab_count = {}
    self.n_vocab = len(self.vocab2index)
  
  def add_vocab(self, sentence):
    for word in str(sentence).split(" "):
      if word not in self.vocab2index:
        self.vocab2index[word] = self.n_vocab
        self.vocab_count[word] = 1
        self.index2vocab[self.n_vocab] = word
        self.n_vocab += 1
      else:
        self.vocab_count[word] += 1

In [6]:
#길이 filtering
def filter_pair(pair, source_max_length, target_max_length):
  return len(pair[0].split(' ')) < source_max_length and len(pair[1].split(' ')) < target_max_length

In [7]:
source_vocab = Vocab()
target_vocab = Vocab()
def preprocess(corpus, source_max_length, target_max_length):
  print('reading corpus...')
  pairs = []
  
  #for line in corpus:
    #pairs.append([s for s in line.strip().lower().split('\t')])
  #print('Read {} sentence pairs'.format(len(pairs)))

  #pairs = [pair for pair in pairs if filter_pair(pair, source_max_length, target_max_length)]
  #print('Trimed to {} sentence pairs'.format(len(pairs)))
  pairs = corpus

  print('Counting Words...')
  for pair in pairs:
    source_vocab.add_vocab(pair[0])
    target_vocab.add_vocab(pair[1])
  print('source vocab size = ', source_vocab.n_vocab)
  print('target vocab size = ', target_vocab.n_vocab)

  return pairs, source_vocab, target_vocab

In [8]:
def concat_vocab(voca1, sentence):
  voca1.add_vocab(sentence)
  return voca1

In [9]:
class Encoder(nn.Module):
  def __init__(self, input_size, hidden_size):
    super(Encoder, self).__init__()
    self.hidden_size = hidden_size
    self.embedding = nn.Embedding(input_size, hidden_size)
    self.gru = nn.GRU(hidden_size, hidden_size)

  def forward(self, x, hidden):
    x = self.embedding(x).view(1, 1, -1)
    x, hidden = self.gru(x, hidden)
    return x, hidden

In [10]:
class Decoder(nn.Module):
  def __init__(self, hidden_size, output_size):
    super(Decoder, self).__init__()
    self.hidden_size = hidden_size
    self.embedding = nn.Embedding(output_size, hidden_size)
    self.gru = nn.GRU(hidden_size, hidden_size)
    self.out = nn.Linear(hidden_size, output_size)
    self.softmax = nn.LogSoftmax(dim = 1)

  def forward(self, x, hidden):
    x = self.embedding(x).view(1, 1, -1)
    x, hidden = self.gru(x, hidden)
    x = self.softmax(self.out(x[0]))
    return x, hidden

In [11]:
#sentence -> index tensor(텐서로 변환)
def tensorize(vocab, sentence):
  indexes = [vocab.vocab2index[word] for word in str(sentence).split(' ')]
  indexes.append(vocab.vocab2index["<EOS>"])
  return torch.Tensor(indexes).long().to(device).view(-1, 1)

In [12]:
#train seq2seq
def train(pairs, source_vocab, target_vocab, encoder, decoder, n_iter,
          print_every = 1000, learning_rate = 0.01):
  loss_total = 0

  encoder_optimizer = optim.Adam(encoder.parameters(), lr = learning_rate)
  decoder_optimizer = optim.Adam(decoder.parameters(), lr = learning_rate)

  training_batch = [random.choice(pairs) for _ in range(n_iter)]
  training_source = [tensorize(source_vocab, pair[0]) for pair in training_batch]
  training_target = [tensorize(target_vocab, pair[1]) for pair in training_batch]

  criterion = nn.CrossEntropyLoss()

  for i in range(1, n_iter + 1):
    source_tensor = training_source[i-1]
    target_tensor = training_target[i-1]

    encoder_hidden = torch.zeros([1, 1, encoder.hidden_size]).to(device)

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    source_length = source_tensor.size(0)
    target_length = target_tensor.size(0)

    loss = 0

    for enc_input in range(source_length):
      _, encoder_hidden = encoder(source_tensor[enc_input], encoder_hidden)

    decoder_input = torch.Tensor([[SOS_token]]).long().to(device)
    decoder_hidden = encoder_hidden

    for di in range(target_length):
      decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
      loss += criterion(decoder_output, target_tensor[di])
      decoder_input = target_tensor[di] #teacher forcing
    #teacher forcing = output값을 다음의 인코더로 넣어줄 수 있지만
    #연산량을 줄이기 위해 정답을 넣어준다. 각각 장단점이 있기에
    #티처 포싱의 단점을 막기 위해 랜덤으로 x%만 티처 포싱을 진행한다.

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    loss_iter = loss.item() / target_length
    loss_total += loss_iter

    if i % print_every == 0:
      loss_avg = loss_total / print_every
      loss_total = 0
      print('[{} - {}%] loss = {:05.4f}'.format(i, i/n_iter*100, loss_avg))

In [13]:
def evaluate(pairs, source_vocab, target_vocab, encoder, decoder, target_max_length):
  for pair in pairs:
    print('>', pair[0])
    print('=', pair[1])
    source_tensor = tensorize(source_vocab, pair[0])
    source_length = source_tensor.size()[0]
    encoder_hidden = torch.zeros([1, 1, encoder.hidden_size]).to(device)

    for ei in range(source_length):
      _, encoder_hidden = encoder(source_tensor[ei], encoder_hidden)

    decoder_input = torch.Tensor([[SOS_token]]).long().to(device)
    decoder_hidden = encoder_hidden
    decoded_words = []

    for di in range(target_max_length):
      decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
      _, top_index = decoder_output.data.topk(1)
      if top_index.item() == EOS_token:
        decoded_words.append('<EOS>')
        break
      else:
        decoded_words.append(target_vocab.index2vocab[top_index.item()])
      
      decoder_input = top_index.squeeze().detach()

    predicted_words = decoded_words
    predict_sentence = " ".join(predicted_words)
    print('<', predict_sentence)
    print("")

In [14]:
SOURCE_MAX_LENGTH = 100
TARGET_MAX_LENGTH = 100

In [15]:
load_pairs, load_source_vocab, load_target_vocab = preprocess(data_pair, SOURCE_MAX_LENGTH,
                                                              TARGET_MAX_LENGTH)
print(load_pairs)

업이면 당연히 같이여'], ['전과 이중전공 신청 전과 선택 이중전공 신청 아니', "가능.', '아진짜요?다행이다ㅠㅠㅠㅠ감사합니다!! 근데 이런 건 어디서 확인할 수 있나요..ㅠ', '학종지에 물어보심 됩니다.', '학종지가 뭔지 여쭤봐도 될까요..? 새내기라 모르겠어요 ㅠㅠ', '학사종합지원센터요.  학교에서 학사관련 담당하는 부서', '감사합니다!"], ['이중전공 대하 고민 머리 터지 학기 학점 35 국제 학부 국제 통상 정치 외교 가망 국제 통상 시험 영어 수학 나오 정치 외교 면접 정치 시사 관하 질문 나오 준비 모르 신문 사설 국제 배우 국제 통상 경제인 나중 영사 시험 생각 도움 선배', '3.5였는데 2학기때 학점 잘받으면 되지않을까'], ['이중전공 경제 통계 전공 언어 고등학교 수학 좋아하', "설캠에서 집이 가깝고 학점이 높으시면 경제하시고', '글캠에서 집이 가깝고 학점이 좀 낮으시면 통계하세용', '감사합니다!!"], ['나중 복전 이중전공 올리', "복전에서 이중은 무슨말이얌?', '말 그대로 처음엔 복전을 했다가 나중에 이중전공 시험을 쳐서 이중으로 올리는 거요’', '부전공말하는거지..?', '억 네', '이중변경공지에 티오 직접봐', '헉 그거는 국통 홈페이지에서 확인 가능한가요? 이중시험 공고만 봤더니...', '학사공지 이중변경공지 첨부파일', '앗 감사합니다!', '별로 안나는데 그렇게 메리트가 잇을까도싶궁..1전공이 상경이라', '음 저는 1전공이 언어라서욥... 상경이시면 안하셔도 되겠네요"], ['새내기 학점 관리 설캠 이중전공 못하 설캠 부전공 나중 이중전공 바꾸 힘들', "베트남어 빼고는 미달인데... 설캠생활하고 싶으면 그냥 아무 언어과 지원하시면 걍 붙어요', '아그렇구나 알려주셔서 감사해요 !', '음 근데 설캠 왔다갔다하시기 힘드시지 않아요..? 거리도 있고 힘드실 것 같은데 언어과라면 글캠에도 통번역 있으니까 그쪽 알아봐도 될 것 같은데....', '제가 언어전공이라서 언어말고 다른 전공을 알아보는 중이였

In [16]:
enc_hidden_size = 30
dec_hidden_size = enc_hidden_size
enc = Encoder(load_source_vocab.n_vocab, enc_hidden_size).to(device)
dec = Decoder(dec_hidden_size, load_target_vocab.n_vocab).to(device)

In [17]:
train(load_pairs, load_source_vocab, load_target_vocab, enc, dec, 50000, print_every = 1000)

[1000 - 2.0%] loss = 8.3413


KeyboardInterrupt: 

In [0]:
evaluate(load_pairs, load_source_vocab, load_target_vocab, enc, dec, TARGET_MAX_LENGTH)

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
= 횟수 상의 제한은 없습니다~
< 네 가능합니다 <EOS>

> 질문 드리 학기 18 학점 학기 20 학점 수강 학점 짜리 학년 사실 36 학점 채우 수강편람 학년 34 학점 미만 학기 기말고사 과목 재수강 과목 재수강 학기 이수 학점 줄어들 개념 재수강 인하 학년 이수 학점 34 학점 미만 경우 어떻 처리 경우 문제 감사
= 학년별로 이수해야할 학점이 있긴 한데 사실상 형식적인 겁니다 1학년 성적이 모두 F라서 이수학점이 없든 재수강으로 당시 이수한 학점이 조금씩 차감되든 전혀 상관 없으며, 졸업전까지 모든 졸업학점을 다 채우시면 됩니다~', '만일 재수강으로 1학년 당시 이수학점이 34학점 미만이 되어 문제가 생긴다면 저도 졸업할 수 없어요 ㅋㅋ ㅜ', '으앙ㅋㅋㅋㅋ그렇구나 글을 두서없이 써서 조교님이 알아들으실 수 있을까 했는데  역시 개떡같이 말해도 찰떡같이 알아들어주시는군여?? 이번에도 답변 감사합니당
< 네 그렇게 됩니다 <EOS>

> 학기 이중전공 시험 떨어지 경영 후기 이중
= 이중전공 신청, 즉 재학생의 이중전공 변경은 4학년때까지 가능합니다 후기이중으로 올리는 방법도 있긴 한데 그건 별도의 심사과정을 거칩니다(신청자의 졸업이수학점 충족 등) 따라서 현재 이수한 학점, 추가학기 등록 여부를 고려해 계획을 세우시길 바랍니다
< 이중전공 변경은 있던데 2학년 이후부턴 또는 매학기마다 학년별, 이중전공 변경신청으로만 배정이 가능합니다 <EOS>

> 영어 학과 전과 경영학 돌리 이중전공 가능
= eicc가 영어학과 내 전공과정이라면 불가능합니다 전과생은 전적학과의 전공을 이중으로 돌릴 수 없거든요', '영어대학 안에 eicc 영어 영미문학 이렇게 세개가 있는데 저 그럼 완전 뷸가한건가요?', '학과 내 전공이 아니라 단과대라면 사정이 다를 것 같네요...전적학과 전공만 아니면 되거든요 이건 학종지에 한번 문의해보셔야 할 듯 합니다..설캠 쪽 세부전공은 저도 잘 몰라서요 ㅜㅜ', '알

In [0]:
new_pair = [['이중 전공 신청 경쟁률 확인', '']]

load_new_pair, load_new_source_vocab, load_new_target_vocab = preprocess(new_pair, SOURCE_MAX_LENGTH,
                                                              TARGET_MAX_LENGTH)
source_vocab_reload = concat_vocab(source_vocab, new_pair[0])

evaluate(load_new_pair, source_vocab_reload, load_target_vocab, enc, dec, TARGET_MAX_LENGTH)

reading corpus...
Counting Words...
source vocab size =  3815
target vocab size =  29255
> 이중 전공 신청 경쟁률 확인
= 
< 본인의 졸업학점을 다 채우시면 됩니다 <EOS>

